<a href="https://colab.research.google.com/github/juanxo90/cargamos/blob/main/cargamos_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba técnica para la posición de data science en Cargamos



In [1]:
# importamos las librerias
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
sns.set_theme(style="whitegrid")

import ipywidgets as widgets
from ipywidgets import AppLayout, GridspecLayout, Layout, HTML, VBox, Label

In [2]:
pd.set_option("display.max_columns", None)

# cargamos los datos
paq_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/paquete_clean.csv")
reg_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/register_clean.csv")

In [3]:
# hacemos una join en un solo dataframe para poder trabajar los datos de forma conjunta
df = reg_det.merge(paq_det, how="inner", on=["id_paquete"])

In [4]:
# agrupamos por entregas a partir del estado y su respectivo complemento
grouped = df[["arreglo_estado", "complemento_estado", "estado"]].groupby(["arreglo_estado", "complemento_estado", "estado"])["estado"].count()
grouped = pd.DataFrame(grouped)
grouped = grouped.rename(columns={'estado': 'conteo'})
grouped.reset_index(inplace=True)

In [5]:
# tomamos los datos que son fallidos y agrupamos por cedis y por fecha
df_ns = df[df.estado != "SUCCESSFUL"].reset_index(drop=True)
df_ns.cedis = df_ns.cedis.str.upper()

df_ns["hora_falla"] = pd.to_datetime(df_ns.hora_falla)
grouped_ns =  df_ns.groupby([df_ns.hora_falla.dt.date, df_ns.cedis])["hora_falla"].count()
grouped_ns  = pd.DataFrame(grouped_ns)
grouped_ns = grouped_ns.rename(columns={'hora_falla': 'total'})
grouped_ns = grouped_ns.reset_index()
grouped_ns.index =grouped_ns.hora_falla

In [26]:
# Función para desplegar el bar plot
def plot_bar(comp_edo):
    """
    plot barplor
    """
    fig = px.bar(grouped[grouped.complemento_estado == comp_edo], 
                x='arreglo_estado', y='conteo',  color='estado', barmode='group',
                labels={
                        "estado": "Estado",
                        "conteo": "Totales",
                        "arreglo_estado": "Arreglo estado"
                    },
                title='Gráfico del estado en la entrega para {}'.format(comp_edo),
                log_y=True, width=650, height=800)
    fig.show()
    
# funcion para analizar los CEDIS por fecha
def plot_fail(cedis):
    total = grouped_ns[grouped_ns.cedis == cedis].total.sum()
    fig = px.line(width=600, height=300)
    fig.add_scatter(x=grouped_ns[grouped_ns.cedis == cedis].index, 
                    y=grouped_ns[grouped_ns.cedis == cedis].total)
    fig.update_xaxes(rangeslider_visible=False, 
                     dtick="d1", 
                     tickformat="%d/%m/%y")
    fig.update_layout(
        xaxis_title="Día",
        yaxis_title="# Entregas fallidas", 
        title='Entregas fallidas por día para el cedis {} <br> Total de entregas fallidas {}'.format(cedis, total))

    fig.show()

In [27]:
l = HTML("<h1> <center> BI análisis de datos entregados </h1>", layout=Layout(height='auto'))
it1 = widgets.Dropdown(options=grouped.complemento_estado.unique(),
                       description='Complemento <br>estado:',disabled=False,
                       layout=Layout(width='300px', height='50px'))
it2 = widgets.Dropdown(options=grouped_ns.cedis.unique(),
                       description='CEDIS:',disabled=False,
                       layout=Layout(width='300px', height='50px'))

interactive_bp= widgets.interactive_output(plot_bar, {"comp_edo":it1})
interactive_cedis = widgets.interactive_output(plot_fail, {"cedis":it2})

l2 = HTML("""
          <h4>
          A partir de las gráficas de barras tenemos que algunos "DELIVERED_ATTEMPED" presentan 
          un estado "SUCESSFUL", siendo esta una de las primeras razones para analizar los datos
          para verificar que estos se han ido actualizado de forma continua, son errores humanos 
          o si existe algun retrazo en las actualizaciones. Fue necesario el uso de escala logaritmica 
          para mostrar las gráficas pues las entregas "SUCESSFUL" han sido aquellas las que en mayor 
          propoción tenemos a pesar de tener una etiqueta que esta relacionada con una entrega fallida.<br> <br>
          Contario a lo que se podría pensar, los fines de semana (como lo muestra la serie de tiempo)
          son aquellos días donde se tienen un mayor número de entregas fallidas. Lo que se podría 
          realizar es el análisis de una serie de tiempo con un mayor número de datos con entregas fallidas 
          para conocer si existe una tendencia. Teniendo estos datos analizar si es posible realizar una 
          estrategia para reducir el número de entregas fallidas, siempre y cuando la proporción afecte 
          la reputación de la compañía. <br><br>
          A partir de los datos es importante conocer si la localización geográfica juega un papel en que 
          se tengan entregas fallidas para poder preparar una posible logistica de entregas, y analizar el 
          porque una base de datos podría estar desactualizada con respecto a la otra.
          </h4>
          """, layout=Layout(height='auto'))


grid = GridspecLayout(12,2,grid_gap = "10 px", height='auto' ,width='auto')

grid[1:, 0] = interactive_bp
grid[1:, 0].layout.height = 'auto'
grid[0, 0] = it1
grid[0, 1] = it2
grid[1:6, 1] = interactive_cedis
grid[6:, 1] = l2

# grid[0, 3:] = interactive_box



display(l,grid)

HTML(value='<h1> <center> BI análisis de datos entregados </h1>', layout=Layout(height='auto'))

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', height='auto')), Dropdown(description='Co…